In [1]:
import torch
import pandas as pd
import random
import os
from shutil import copyfile

In [11]:
# os.mkdir('./working/data')
os.mkdir('./working/data/images')
os.mkdir('./working/data/labels')

os.mkdir('./working/data/images/train')
os.mkdir('./working/data/images/valid')

os.mkdir('./working/data/labels/train')
os.mkdir('./working/data/labels/valid')

In [12]:
with open('./working/new_train_yaml', 'w+') as file:
    file.write(
        """
        # parameters
        nc: 1  # number of classes
        depth_multiple: 0.33  # model depth multiple
        width_multiple: 0.50  # layer channel multiple

        # anchors
        anchors:
          - [10,13, 16,30, 33,23]  # P3/8
          - [30,61, 62,45, 59,119]  # P4/16
          - [116,90, 156,198, 373,326]  # P5/32

        # YOLOv5 backbone
        backbone:
          # [from, number, module, args]
          [[-1, 1, Focus, [64, 3]],  # 0-P1/2
           [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
           [-1, 3, BottleneckCSP, [128]],
           [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
           [-1, 9, BottleneckCSP, [256]],
           [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
           [-1, 9, BottleneckCSP, [512]],
           [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
           [-1, 1, SPP, [1024, [5, 9, 13]]],
           [-1, 3, BottleneckCSP, [1024, False]],  # 9
          ]

        # YOLOv5 head
        head:
          [[-1, 1, Conv, [512, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 6], 1, Concat, [1]],  # cat backbone P4
           [-1, 3, BottleneckCSP, [512, False]],  # 13

           [-1, 1, Conv, [256, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 4], 1, Concat, [1]],  # cat backbone P3
           [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

           [-1, 1, Conv, [256, 3, 2]],
           [[-1, 14], 1, Concat, [1]],  # cat head P4
           [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

           [-1, 1, Conv, [512, 3, 2]],
           [[-1, 10], 1, Concat, [1]],  # cat head P5
           [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

           [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
          ]
        """
    )

In [13]:
with open('./working/new_data_yaml', 'w+') as file:
    file.write(
        """
        train: ./working/data/images/train
        val: ./working/data/images/valid

        nc: 1
        names: ['cans']
        """
    )

In [62]:
!python train.py --img 400 --batch 10 --epochs 10 --data './working/new_data_yaml' --cfg './models/yolov5s.yaml' --weights 'last_yolov5s_results.pt' --name joos --nosave --cache

train: weights=last_yolov5s_results.pt, cfg=./models/yolov5s.yaml, data=./working/new_data_yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=10, imgsz=400, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, evolve=None, bucket=, cache_images=True, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=joos, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1
github: ⚠️ WARNING: code is out of date by 26 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 🚀 v5.0-301-g0cc7c58 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0,

In [46]:
!python detect.py --source './test_1318.jpg' --weights './runs/train/joos16/weights/best.pt' --img 416 --conf 0.20 --save-txt

detect: weights=['./runs/train/joos16/weights/best.pt'], source=./test_1318.jpg, imgsz=416, conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-301-g0cc7c58 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
image 1/1 /h

In [1]:
import os

In [9]:
(os.listdir('./working/data/labels/valid'))

['test_1318.txt',
 'test_1340.txt',
 'test_1400.txt',
 'test_1337.txt',
 'test_1379.txt',
 '.ipynb_checkpoints',
 'test_1371.txt']

In [10]:
(os.listdir('./working/data/images/valid'))

['test_1371.jpg',
 'test_1340.jpg',
 'test_1337.jpg',
 'test_1318.jpg',
 'test_1379.jpg',
 '.ipynb_checkpoints',
 'test_1400.jpg']

In [36]:
##write custom model .yaml
#you can configure this based on other YOLOv5 models in the models directory
with open('./models/custom_yolov5s.yaml', 'w') as f:
    # parameters

    f.write('nc: ' + str(1) + '\n')
    #f.write('nc: ' + str(len(class_labels)) + '\n')
    f.write('depth_multiple: 0.33'  + '\n') # model depth multiple
    f.write('width_multiple: 0.50'  + '\n')  # layer channel multiple
    f.write('\n')
    f.write('anchors:' + '\n')
    f.write('  - [10,13, 16,30, 33,23] ' + '\n')
    f.write('  - [30,61, 62,45, 59,119]' + '\n')
    f.write('  - [116,90, 156,198, 373,326] ' + '\n')
    f.write('\n')

    f.write('backbone:' + '\n')
    f.write('  [[-1, 1, Focus, [64, 3]],' + '\n')
    f.write('   [-1, 1, Conv, [128, 3, 2]],' + '\n')
    f.write('   [-1, 3, Bottleneck, [128]],' + '\n')
    f.write('   [-1, 1, Conv, [256, 3, 2]],' + '\n')
    f.write('   [-1, 9, BottleneckCSP, [256]],' + '\n')
    f.write('   [-1, 1, Conv, [512, 3, 2]], ' + '\n')
    f.write('   [-1, 9, BottleneckCSP, [512]],' + '\n')
    f.write('   [-1, 1, Conv, [1024, 3, 2]],' + '\n')
    f.write('   [-1, 1, SPP, [1024, [5, 9, 13]]],' + '\n')
    f.write('   [-1, 6, BottleneckCSP, [1024]],' + '\n')
    f.write('  ]' + '\n')
    f.write('\n')

    f.write('head:'  + '\n')
    f.write('  [[-1, 3, BottleneckCSP, [1024, False]],'  + '\n')
    f.write('   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1, 0]],' + '\n')
    f.write('   [-2, 1, nn.Upsample, [None, 2, "nearest"]],' + '\n')

    f.write('   [[-1, 6], 1, Concat, [1]],' + '\n')
    f.write('   [-1, 1, Conv, [512, 1, 1]],' + '\n')
    f.write('   [-1, 3, BottleneckCSP, [512, False]],' + '\n')
    f.write('   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1, 0]],' + '\n')

    f.write('   [-2, 1, nn.Upsample, [None, 2, "nearest"]],' + '\n')
    f.write('   [[-1, 4], 1, Concat, [1]],' + '\n')
    f.write('   [-1, 1, Conv, [256, 1, 1]],' + '\n')
    f.write('   [-1, 3, BottleneckCSP, [256, False]],' + '\n')
    f.write('   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1, 0]],' + '\n')
    f.write('\n' )
    f.write('   [[], 1, Detect, [nc, anchors]],' + '\n')
    f.write('  ]' + '\n')

print('custom model config written!')


custom model config written!
